In [250]:
from pathlib import Path
import numpy as np
import iris
from aeolus.calc import wind_rot_div
from aeolus.core import AtmoSim
from aeolus.model import um
tgt_plevs = np.logspace(7, 2, 65)  # target pressure levels
from aeolus.coord import interp_cube_from_height_to_pressure_levels
import warnings
import matplotlib.pyplot as plt
from aeolus.plot import subplot_label_generator

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import numpy as np
from matplotlib import animation

from matplotlib.offsetbox import AnchoredText




In [251]:
tgt_plevs[64]

100.0

In [252]:
warnings.filterwarnings("ignore", module="iris")

In [253]:
#this is for the 10x solar runs
# parents_dir_k1 = Path("/data/mz355/um_runs/wasp96b/kinetics/u-cp805/raw/atmos_base_0080")
parents_dir_eq1 = Path("/data/mz355/um_runs/wasp96b/equilibrium/u-cp770/raw/")

# parents_dir = Path.home().parent.parent/"data" / "mz355" /"um_runs"/"wasp96b"/"equilibrium"/"u-cp158"/"raw"
flist = []
for windfile in parents_dir_eq1.glob('**/atmos_base_*/atmosa.*'):  # this directory and all subdirectories, recursively
     if ("atmosa.pc" in windfile.name or "atmosa.pb" in windfile.name):
        flist.append(str(windfile))

len(sorted(flist))

print(sorted(flist))
# print(parents_dir)

# eq_file_1 = parents_dir_eq1 / "atmosa.pc000000120_00"
# eq_pressure_1 = parents_dir_eq1 / "atmosa.pb000000120_00"
# eq_data_1 = iris.load(eq_file_1)
# eq_pressure_file_1 = iris.load(eq_pressure_1)


# kin_file_1 = parents_dir_k1 / "atmosa.pc00000070_00"
# kin_pressure_1 = parents_dir_k1 / "atmosa.pb00000070_00"

# kin_data_1 = iris.load(kin_file_1)
# print(kin_data_1)
# print(eq_data_1)

['/data/mz355/um_runs/wasp96b/equilibrium/u-cp770/raw/atmos_base_0010/atmosa.pb00000000_00', '/data/mz355/um_runs/wasp96b/equilibrium/u-cp770/raw/atmos_base_0010/atmosa.pc00000000_00', '/data/mz355/um_runs/wasp96b/equilibrium/u-cp770/raw/atmos_base_0020/atmosa.pb00000010_00', '/data/mz355/um_runs/wasp96b/equilibrium/u-cp770/raw/atmos_base_0020/atmosa.pc00000010_00', '/data/mz355/um_runs/wasp96b/equilibrium/u-cp770/raw/atmos_base_0030/atmosa.pb00000020_00', '/data/mz355/um_runs/wasp96b/equilibrium/u-cp770/raw/atmos_base_0030/atmosa.pc00000020_00', '/data/mz355/um_runs/wasp96b/equilibrium/u-cp770/raw/atmos_base_0040/atmosa.pb00000030_00', '/data/mz355/um_runs/wasp96b/equilibrium/u-cp770/raw/atmos_base_0040/atmosa.pc00000030_00', '/data/mz355/um_runs/wasp96b/equilibrium/u-cp770/raw/atmos_base_0050/atmosa.pb00000040_00', '/data/mz355/um_runs/wasp96b/equilibrium/u-cp770/raw/atmos_base_0050/atmosa.pc00000040_00', '/data/mz355/um_runs/wasp96b/equilibrium/u-cp770/raw/atmos_base_0060/atmosa.pb0

In [254]:
winds = iris.load(flist)

In [255]:
print(winds)

0: air_pressure / (Pa)                 (time: 17; model_level_number: 67; latitude: 90; longitude: 144)
1: air_temperature / (K)               (time: 17; model_level_number: 66; latitude: 90; longitude: 144)
2: upward_air_velocity / (m s-1)       (time: 17; model_level_number: 67; latitude: 90; longitude: 144)
3: x_wind / (m s-1)                    (time: 17; model_level_number: 66; latitude: 90; longitude: 144)
4: y_wind / (m s-1)                    (time: 17; model_level_number: 66; latitude: 91; longitude: 144)


In [256]:
#sorting out the pressure files

pressure_cube_eq1 = winds.extract_cube("air_pressure")[:,:66,:,:]
# pressure_cube_eq1=pressure_cube_eq1

In [257]:
pressure_cube_eq1

Air Pressure (Pa),time,model_level_number,latitude,longitude
Shape,17,66,90,144
Dimension coordinates,,,,
time,x,-,-,-
model_level_number,-,x,-,-
latitude,-,-,x,-
longitude,-,-,-,x
Auxiliary coordinates,,,,
forecast_period,x,-,-,-
level_height,-,x,-,-
sigma,-,x,-,-


In [258]:
#equilibrium first

my_run = AtmoSim(
    cubes=winds,
    name="t1e",
    description="Time-mean output from a UM simulation of the THAI Hab 1 case.",
    planet="WASP-96b",  # this reads constants from a JSON file
)
print(my_run)

aeolus.core AtmoSim 't1e' [5 cubes]
description | Time-mean output from a UM simulation of the THAI Hab 1 case.
     planet | WASP-96b
      model | Model with 95 fields: t, fcst_ref, fcst_prd, z, lev, s, d, p, y, x, u, v, w, pres, thta, exner, sh,
t_sfc, p_sfc, toa_isr, toa_olr, toa_olr_cs, toa_osr, toa_osr_cs, sfc_dn_lw, sfc_dn_lw_cs, sfc_dn_sw,
sfc_dn_sw_cs, sfc_net_down_lw, sfc_net_down_sw, lw_up, lw_up_forcing, lw_dn_forcing, sw_up, lw_dn,
sw_dn, lw_up_cs, sw_up_cs, lw_dn_cs, sw_dn_cs, sfc_shf, sfc_lhf, sfc_evap, sfc_x_wind_stress,
sfc_y_wind_stress, temp, dens, ghgt, rh, cld_ice_mf, cld_liq_mf, rain_mf, cca_anvil, ccw_rad,
cld_top_hgt, cld_ice_v, cld_liq_v, cld_v, caf, caf_h, caf_m, caf_l, caf_vl, ppn, ls_rain, ls_snow,
cv_rain, cv_snow, dt_sw, dt_sw_cs, dt_lw, dt_lw_cs, dt_lsppn, dt_bl, dt_cv, dt_lscld, dt_adv, dq_sw,
dq_lw, dq_lsppn, dq_bl, dq_cv, dq_lscld, dq_adv, du_bl, du_cv, du_solve, du_adv, du_total,
soil_moist, ep_flux_y, ep_flux_z, ep_flux_div, light_flash_rate, light_n

In [259]:
u = my_run.u.extract(iris.Constraint())
v = my_run.v.extract(iris.Constraint())
v = v.regrid(u, iris.analysis.Linear())

In [260]:
u_plevs = interp_cube_from_height_to_pressure_levels(u, pressure_cube_eq1, tgt_plevs)
v_plevs = interp_cube_from_height_to_pressure_levels(v, pressure_cube_eq1, tgt_plevs)
print(u_plevs.shape, v_plevs.shape)

(17, 65, 90, 144) (17, 65, 90, 144)


In [261]:
# u_at_8km = my_run.u.extract(iris.Constraint(**{um.z: 312200}))
# v_at_8km = my_run.v.extract(iris.Constraint(**{um.z: 80000}))

In [262]:
# print(u_at_8km.coord(um.z))

In [263]:
rot_div_cubes = wind_rot_div(u_plevs[:,64], v_plevs[:,64])
print(rot_div_cubes)

{'u_total': <iris 'Cube' of x_wind / (m s-1) (time: 17; latitude: 90; longitude: 144)>, 'v_total': <iris 'Cube' of y_wind / (m s-1) (time: 17; latitude: 90; longitude: 144)>, 'u_div': <iris 'Cube' of irrotational_eastward_wind / (m s**-1) (time: 17; latitude: 90; longitude: 144)>, 'v_div': <iris 'Cube' of irrotational_northward_wind / (m s**-1) (time: 17; latitude: 90; longitude: 144)>, 'u_rot': <iris 'Cube' of non_divergent_eastward_wind / (m s**-1) (time: 17; latitude: 90; longitude: 144)>, 'v_rot': <iris 'Cube' of non_divergent_northward_wind / (m s**-1) (time: 17; latitude: 90; longitude: 144)>, 'u_rot_zm': <iris 'Cube' of zonal_mean_of_non_divergent_eastward_wind / (m s**-1) (time: 17; latitude: 90; longitude: 144)>, 'u_rot_eddy': <iris 'Cube' of zonal_deviation_of_non_divergent_eastward_wind / (m s**-1) (time: 17; latitude: 90; longitude: 144)>, 'v_rot_zm': <iris 'Cube' of zonal_mean_of_non_divergent_northward_wind / (m s**-1) (time: 17; latitude: 90; longitude: 144)>, 'v_rot_edd

In [264]:
WIND_CMPNT_META = {
    "total": {"title": "Total", "qk_ref_wspd": 60, "kw_plt": {"color": "#222222"}},
    "div": {"title": "Divergent", "qk_ref_wspd": 10, "kw_plt": {"color": "C0"}},
    "rot_zm": {"title": "Jet", "qk_ref_wspd": 60, "kw_plt": {"color": "C3"}},
    "rot_eddy": {"title": "Eddy", "qk_ref_wspd": 30, "kw_plt": {"color": "C2"}},
}

In [265]:
lons = (my_run.coord.x.points)-180
lats = my_run.coord.y.points
print(lons.shape, lats.shape)
# Equivalent to
# lons = u_at_8km.coord("longitude").points
# lats = u_at_8km.coord("latitude").points

xstride = 8
ystride = 6
xsl = slice(None, None, xstride)
ysl = slice(None, None, ystride)

(144,) (90,)


In [266]:
fig = plt.figure(figsize=(20, 10))
plt.suptitle("Equilibrium 10x Solar Metallicity")
def animate(i: int):
    mosaic = [[*WIND_CMPNT_META.keys()][:2], [*WIND_CMPNT_META.keys()][2:]]
    print(i)
    axd = fig.subplot_mosaic(mosaic)
    iletters = subplot_label_generator()
    for wind_key, wind_meta in WIND_CMPNT_META.items():
        ax = axd[wind_key]
        ax.clear()
        ax.set_title(f"({next(iletters)})", loc="left")
        ax.set_title(wind_meta['title'], loc="center")
        ax.set_ylim(-90, 90)
        ax.set_xlim(-180, 180)
        at = AnchoredText(f"day: {i * 10}",
                      loc='upper left', prop=dict(size=8), frameon=True,
                      )
        ax.add_artist(at)
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel("Latitude [$\degree$]")
        if ax.get_subplotspec().is_last_row():
            ax.set_xlabel("Longitude [$\degree$]")
        # print(_u_cmpnt = rot_div_cubes[f"u_{wind_key}"].data.shape)

        _u_cmpnt = rot_div_cubes[f"u_{wind_key}"].data[i,:,:]
        _v_cmpnt = rot_div_cubes[f"v_{wind_key}"].data[i,:,:]

        qv = ax.quiver(lons[xsl], lats[ysl], _u_cmpnt[ysl, xsl], _v_cmpnt[ysl, xsl], **wind_meta["kw_plt"])
    return qv,

<Figure size 1440x720 with 0 Axes>

In [267]:
ani = animation.FuncAnimation(fig, animate, interval=200, blit=False, repeat=False, frames=23, save_count=10000)    


In [268]:
ani.save("eq_mdh1.gif", writer='imagemagick',fps=10)

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


IndexError: index 17 is out of bounds for axis 0 with size 17